<a href="https://colab.research.google.com/github/KaifFaiyaz/Classification_gnn/blob/main/PyGeom1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

In [2]:
print("Hello World")

Hello World
